In [ ]:
import json
import pandas as pd

from glob import glob
from concurrent.futures import ThreadPoolExecutor

data_root = 'data/gamepress/pokemons'

def process_pokemon_file(pokemon_file):
	try:
		with open(pokemon_file) as f:
			pokemon_data = json.load(f)
			
			weaknesses = pokemon_data.get('weaknesses', [])
			# resistances = pokemon_data.get('resistances', [])
			# weakness_str = ','.join([_['type'] for _ in weaknesses])
			# not_resistance_str = '&'.join([f'!{_["type"]}' for _ in resistances])
			# search_str = f'{not_resistance_str}&{weakness_str}'

			skills = []
			for i in range(1, 4):
				for _ in [_['type'] for _ in weaknesses]:
					skills.append(f'@{i}{_}')

			search_str = ','.join(skills[:len(weaknesses)]) + '&' + ','.join(skills[len(weaknesses):])

			name = pokemon_data.get('name')
			if name is None:
				name_id = pokemon_data.get('id')
				name = ' '.join([_.capitalize() for _ in name_id.split('-')])

			return {
				'name': name,
				'search': search_str,
				'type': ','.join(pokemon_data.get('types', [])),
				# 'fast_moves': ','.join([_['name'] for _ in pokemon_data.get('fast_moves', [])]),
				# 'charge_moves': ','.join([_['name'] for _ in pokemon_data.get('charge_moves', [])]),
				# 'weaknesses': weakness_str,
				# 'resistances': ','.join([_['type'] for _ in resistances]),
				# 'attack': pokemon_data.get('attack'),
				# 'defense': pokemon_data.get('defense'),
				# 'stamina': pokemon_data.get('stamina'),
				'max_cp': pokemon_data.get('max_cp')
			}
	except Exception as e:
		print(f'Error processing {pokemon_file}. {e}')

with ThreadPoolExecutor() as executor:
	pokemon_files = glob(f'{data_root}/*.json')
	print(f'No. of pokemon files: {len(pokemon_files)}')
	pokemon_datas = list(executor.map(process_pokemon_file, pokemon_files))

pokemons_df = pd.DataFrame(pokemon_datas)

pokemons_df.to_json(f'data/json/individual_pokemon.json', orient='records')
# pokemons_df